In [9]:
import os
import json
from pathlib import Path

import sagemaker
import boto3
from dotenv import load_dotenv

ROOT = Path.cwd().parent.parent

REGION_NAME = "eu-central-1"
ROLE_NAME = "Sagemaker-ExecutionRole"

## Auth

In [10]:
load_dotenv(ROOT / ".env")

HF_KEY = os.getenv("HF_KEY")
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_KEY")
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME

auth_args = {
    "aws_access_key_id": os.getenv("AWS_ID"),
    "aws_secret_access_key": os.getenv("AWS_KEY"),
    "region_name": REGION_NAME,
}

In [11]:
iam = boto3.client("iam", **auth_args)
role = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]

session = sagemaker.Session(boto3.Session(**auth_args))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml


## Deploy

In [12]:
import json
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

hub = {
	'HF_MODEL_ID':'meta-llama/Llama-2-70b-hf',
	'SM_NUM_GPUS': json.dumps(1),
	'HUGGING_FACE_HUB_TOKEN': HF_KEY
}


In [13]:
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.9.3"),
	env=hub,
	role=role, 
)

predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.48xlarge",
	container_startup_health_check_timeout=300,
)
  

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateEndpoint operation: The account-level service limit 'ml.g5.48xlarge for endpoint usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [ ]:
answer = predictor.predict({
	"inputs": "My name is Julien and I like to",
	"parameters": {
		"eos_token_id": 2,
  		"pad_token_id": 2,
  		"max_length": 4096,
		"temperature": 0,
		"do_sample": False,
        "num_return_sequences":1,
	},
})

## Inference

In [ ]:
prompt = "To complete this sentence "

request = {
    "inputs": prompt,
    "parameters": {
        "do_sample": False,
        "temperature": 0,
        "max_new_tokens": 10,
    }  
}

response = predictor.predict(request)

answer = response[0]["generated_text"][len(prompt):]

## Clean up

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()